In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline

In [2]:
import dipy.reconst.dti as dti
import dipy.reconst.dki as dki
import dipy.reconst.cross_validation as xval
import dipy.core.gradients as dpg

In [3]:
import nibabel as nib

In [4]:
import tools
import imp
imp.reload(tools)
from tools import resample_volume

In [15]:
def calc_cod(model, data, mask=None, folds=5):
    pred = xval.kfold_xval(model, data, folds, mask=mask)
    cod = xval.coeff_of_determination(pred, data)
    return cod

In [ ]:
# Replace with a loop over subjects:
subject = '991267'

In [ ]:
label_img = nib.load('/home/ubuntu/data/%s/sess/anat/aparc+aseg.nii.gz'%subject)
resamp_label = resample_volume(label_img, dwi_img)
label_data = resamp_label.get_data()
# Cerebral white matter in both hemispheres + corpus callosum
wm_mask = (label_data==41) | (label_data==2) | (label_data==86)

In [5]:
dwi_img = nib.load('/home/ubuntu/data/%s/sess/dwi/dwi.nii.gz'%subject)
gtab = dpg.gradient_table('/home/ubuntu/data/%s/sess/dwi/dwi.bvals'%subject, 
                          '/home/ubuntu/data/%s/sess/dwi/dwi.bvecs'%subject,
                          b0_threshold=10)

In [ ]:
# Save WM mask upfront, and don't worry about using it for now
label_img = nib.load('/home/ubuntu/data/%s/sess/anat/aparc+aseg.nii.gz'%subject)
resamp_label = resample_volume(label_img, dwi_img)
label_data = resamp_label.get_data()
# Cerebral white matter in both hemispheres + corpus callosum
wm_mask = (label_data==41) | (label_data==2) | (label_data==86)
nib.save(nib.Nifti1Image(wm_mask.astype(int), dwi_img.affine), 'Subject_%s_white_matter_mask.nii.gz'%subject)

In [6]:
data = dwi_img.get_data()

In [14]:
dki_model = dki.DiffusionKurtosisModel(gtab)
dti_model = dti.TensorModel(gtab)

In [16]:
cod_dki = calc_cod(dki_model, data) 
cod_dti = calc_cod(dti_model, data)

/home/ubuntu/source/dipy/dipy/reconst/cross_validation.py:57: RuntimeWarning: divide by zero encountered in true_divide
  return 100 * (1 - (ss_err/ss_tot))
/home/ubuntu/source/dipy/dipy/reconst/cross_validation.py:57: RuntimeWarning: invalid value encountered in true_divide
  return 100 * (1 - (ss_err/ss_tot))


In [ ]:
nib.save(nib.Nifti1Image(cod_dki, dwi_img.affine), 'Subject_%s_dki_COD.nii.gz'%subject)
nib.save(nib.Nifti1Image(cod_dti, dwi_img.affine), 'Subject_%s_dti_COD.nii.gz'%subject)

In [20]:
dki_fit = dki_model.fit(data)
dti_fit = dti_model.fit(data)

In [ ]:
nib.save(nib.Nifti1Image(dki_fit.model_params, dwi_img.affine), 'Subject_%s_dki_model_params.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit.model_params, dwi_img.affine), 'Subject_%s_dti_model_params.nii.gz'%subject)

In [ ]:
nib.save(nib.Nifti1Image(dki_fit.fa, dwi_img.affine), 'Subject_%s_dki_FA.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit.fa, dwi_img.affine), 'Subject_%s_dti_FA.nii.gz'%subject)

In [ ]:
nib.save(nib.Nifti1Image(dki_fit.md, dwi_img.affine), 'Subject_%s_dki_MD.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit.md, dwi_img.affine), 'Subject_%s_dti_MD.nii.gz'%subject)

In [ ]:
nib.save(nib.Nifti1Image(dki_fit.mk(), dwi_img.affine), 'Subject_%s_dki_MK.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit.rk(), dwi_img.affine), 'Subject_%s_dti_RK.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit.ak(), dwi_img.affine), 'Subject_%s_dti_AK.nii.gz'%subject)

In [26]:
idx1000 = (gtab.bvals < 1100) | (gtab.bvals <= 5)
idx2000 = ((gtab.bvals > 1100) & (gtab.bvals < 2100 )) | (gtab.bvals <= 5)
idx3000 = (gtab.bvals > 2100) | (gtab.bvals <= 5)

In [27]:
data1000 = data[..., idx1000]
data2000 = data[..., idx2000]
data3000 = data[..., idx3000]
data1000_2000 = data[..., idx1000 + idx2000]
data1000_3000 = data[..., idx1000 + idx3000]
data2000_3000 = data[..., idx2000 + idx3000]

In [28]:
gtab1000 = dpg.gradient_table(gtab.bvals[idx1000], gtab.bvecs[idx1000], b0_threshold=5)
gtab2000 = dpg.gradient_table(gtab.bvals[idx1000], gtab.bvecs[idx1000], b0_threshold=5)
gtab3000 = dpg.gradient_table(gtab.bvals[idx1000], gtab.bvecs[idx1000], b0_threshold=5)
gtab1000_2000 = dpg.gradient_table(gtab.bvals[idx1000 + idx2000], gtab.bvecs[idx1000 + idx2000], b0_threshold=5)
gtab1000_3000 = dpg.gradient_table(gtab.bvals[idx1000 + idx3000], gtab.bvecs[idx1000 + idx3000], b0_threshold=5)
gtab2000_3000 = dpg.gradient_table(gtab.bvals[idx2000 + idx3000], gtab.bvecs[idx2000 + idx3000], b0_threshold=5)

In [29]:
dti_model1000 = dti.TensorModel(gtab1000)
dti_model2000 = dti.TensorModel(gtab2000)
dti_model3000 = dti.TensorModel(gtab3000)
dti_model1000_2000 = dti.TensorModel(gtab1000_2000)
dti_model1000_3000 = dti.TensorModel(gtab1000_3000)
dti_model2000_3000 = dti.TensorModel(gtab2000_3000)

In [30]:
dti_fit1000 = dti_model1000.fit(data1000, mask=wm_mask)
dti_fit2000 = dti_model2000.fit(data2000, mask=wm_mask)
dti_fit3000 = dti_model3000.fit(data3000, mask=wm_mask)
dti_fit1000_2000 = dti_model1000_2000.fit(data1000_2000, mask=wm_mask)
dti_fit1000_3000 = dti_model1000_3000.fit(data1000_3000, mask=wm_mask)
dti_fit2000_3000 = dti_model2000_3000.fit(data2000_3000, mask=wm_mask)

In [ ]:
nib.save(nib.Nifti1Image(dti_fit1000.fa, dwi_img.affine), 'Subject_%s_dti_1000_FA.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit2000.fa, dwi_img.affine), 'Subject_%s_dti_2000_FA.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit3000.fa, dwi_img.affine), 'Subject_%s_dti_3000_FA.nii.gz'%subject)

nib.save(nib.Nifti1Image(dti_fit1000_2000.fa, dwi_img.affine), 'Subject_%s_dti_1000_2000_FA.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit2000_3000.fa, dwi_img.affine), 'Subject_%s_dti_2000_3000_FA.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit1000_3000.fa, dwi_img.affine), 'Subject_%s_dti_1000_3000_FA.nii.gz'%subject)

In [ ]:
nib.save(nib.Nifti1Image(dti_fit1000.md, dwi_img.affine), 'Subject_%s_dti_1000_MD.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit2000.md, dwi_img.affine), 'Subject_%s_dti_2000_MD.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit3000.md, dwi_img.affine), 'Subject_%s_dti_3000_MD.nii.gz'%subject)

nib.save(nib.Nifti1Image(dti_fit1000_2000.md, dwi_img.affine), 'Subject_%s_dti_1000_2000_MD.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit2000_3000.md, dwi_img.affine), 'Subject_%s_dti_2000_3000_MD.nii.gz'%subject)
nib.save(nib.Nifti1Image(dti_fit1000_3000.md, dwi_img.affine), 'Subject_%s_dti_1000_3000_MD.nii.gz'%subject)

In [33]:
dki_model1000_2000 = dki.DiffusionKurtosisModel(gtab1000_2000)
dki_model1000_3000 = dki.DiffusionKurtosisModel(gtab1000_3000)
dki_model2000_3000 = dki.DiffusionKurtosisModel(gtab2000_3000)

In [34]:
dki_fit1000_2000 = dki_model1000_2000.fit(data1000_2000)
dki_fit1000_3000 = dki_model1000_3000.fit(data1000_3000)
dki_fit2000_3000 = dki_model2000_3000.fit(data2000_3000)

In [ ]:
nib.save(nib.Nifti1Image(dki_fit1000_2000.fa, dwi_img.affine), 'Subject_%s_dki_1000_2000_FA.nii.gz'%subject)
nib.save(nib.Nifti1Image(dki_fit2000_3000.fa, dwi_img.affine), 'Subject_%s_dki_2000_3000_FA.nii.gz'%subject)
nib.save(nib.Nifti1Image(dki_fit1000_3000.fa, dwi_img.affine), 'Subject_%s_dki_1000_3000_FA.nii.gz'%subject)

In [ ]:
nib.save(nib.Nifti1Image(dki_fit1000_2000.md, dwi_img.affine), 'Subject_%s_dki_1000_2000_MD.nii.gz'%subject)
nib.save(nib.Nifti1Image(dki_fit2000_3000.md, dwi_img.affine), 'Subject_%s_dki_2000_3000_MD.nii.gz'%subject)
nib.save(nib.Nifti1Image(dki_fit1000_3000.md, dwi_img.affine), 'Subject_%s_dki_1000_3000_MD.nii.gz'%subject)

In [ ]:
nib.save(nib.Nifti1Image(dki_fit1000_2000.mk(), dwi_img.affine), 'Subject_%s_dki_1000_2000_MK.nii.gz'%subject)
nib.save(nib.Nifti1Image(dki_fit2000_3000.mk(), dwi_img.affine), 'Subject_%s_dki_2000_3000_MK.nii.gz'%subject)
nib.save(nib.Nifti1Image(dki_fit1000_3000.mk(), dwi_img.affine), 'Subject_%s_dki_1000_3000_MK.nii.gz'%subject)